# Dataset preparation

Code for post-processed datasets

1. TAO merged / gridded
2. TAO interped to EUC space

## make TAO hi-res zarr dataset

Takes 10m & hourly data and combines them.

Merges in 10m, 25m current meters with the ADCP data where ADCP is not available at these depths

Filenames:
1. `tao_eq_hr_merged_cur.zarr` : merged adcp, cur but not on uniform grid
2. `tao_eq_hr_gridded.zarr` : merged adcp, cur, temp gridded on ADCP grid

In [25]:
import dcpy.interpolate

In [104]:
adcp = pump.obs.read_tao_adcp(freq='hr')
temp = pump.obs.read_eq_tao_temp_hr()
hrcur = pump.obs.read_eq_tao_cur_hr()

# tao = xr.merge([adcp, temp]).chunk({"time": -1, "longitude": 1, "depth": 10})

100%|██████████| 5/5 [00:01<00:00,  4.02it/s]


In [118]:
adcp = adcp.reindex(depth=np.arange(-500, 1, 5.0))

In [119]:
binned_temp = (
    temp.chunk({"time": 20000}).groupby_bins("depth", np.arange(-502.5, 3, 5))
    .mean()
    .rename({"depth_bins": "depth"})
    .assign_coords(depth=np.arange(-500, 0.1, 5))
)

# interp using PCHIP to ADCP grid
interp_temp = dcpy.interpolate.pchip(
    binned_temp.chunk({"time": 1000, "depth": -1}),
    "depth",
    adcp.depth,
    core_dim="depth",
)

/glade/u/home/dcherian/miniconda3/envs/dcpy/lib/python3.6/site-packages/dask/array/core.py:3855: PerformanceWarning: Increasing number of chunks by factor of 15
  **blockwise_kwargs


In [131]:
merged_adcp.depth

<xarray.DataArray 'depth' (depth: 100)>
array([ -25.,  -10.,  -20.,  -15.,    0., -500., -495., -490., -485., -480.,
       -475., -470., -465., -460., -455., -450., -445., -440., -435., -430.,
       -425., -420., -415., -410., -405., -400., -395., -390., -385., -380.,
       -375., -370., -365., -360., -355., -350., -345., -340., -335., -330.,
       -325., -320., -315., -310., -305., -300., -295., -290., -285., -280.,
       -275., -270., -265., -260., -255., -250., -245., -240., -235., -230.,
       -225., -220., -215., -210., -205., -200., -195., -190., -185., -180.,
       -175., -170., -165., -160., -155., -150., -145., -140., -135., -130.,
       -125., -120., -115., -110., -105., -100.,  -95.,  -90.,  -85.,  -80.,
        -75.,  -70.,  -65.,  -60.,  -55.,  -50.,  -45.,  -40.,  -35.,  -30.])
Coordinates:
    latitude  float32 0.0
  * depth     (depth) float64 -25.0 -10.0 -20.0 -15.0 ... -40.0 -35.0 -30.0
Attributes:
    FORTRAN_format:  
    units:           m
    epic_code:       3
    positive:        down
    standard_name:   depth

In [134]:
# merged the 10, 25m current meters with ADCP
# prefer ADCP where available

depth = [-5, -10, -25]  # could add -5?
region = dict(depth=depth)
cur = hrcur.reindex(time=tao.time).sel(**region)
merged_vel = xr.merge(
    [cur.where(adcp.sel(**region).isnull()), adcp.sel(**region)], compat="no_conflicts"
)

# now add the near-surface stuff with the deeper ADCP stuff
merged_adcp = xr.concat(
    [
        adcp[["u", "v"]].sel(depth=slice(None, -26)),
        merged_vel.sortby("depth", "descending"),
        adcp[["u", "v"]].sel(depth=[-20, -15, 0]),
    ],
    dim="depth",
).sortby("depth", "descending")

assert interp_temp.depth.equals(merged_adcp.depth)
# adcp = tao.sel(**region).where(cur.isnull())
# tao_merged = tao.copy().update(merged)
# tao_merged

In [138]:
tao_gridded = xr.merge([interp_temp.to_dataset(name="T"), adcp])
tao_gridded

<xarray.Dataset>
Dimensions:    (depth: 101, longitude: 5, time: 292267)
Coordinates:
  * time       (time) datetime64[ns] 1985-10-01T06:00:00 ... 2019-02-03
  * depth      (depth) float64 -500.0 -495.0 -490.0 -485.0 ... -10.0 -5.0 0.0
    latitude   float32 0.0
  * longitude  (longitude) float64 -204.0 -195.0 -170.0 -140.0 -110.0
Data variables:
    T          (time, longitude, depth) float64 dask.array<chunksize=(1000, 2, 101), meta=np.ndarray>
    u          (time, depth, longitude) float32 dask.array<chunksize=(292267, 101, 1), meta=np.ndarray>
    v          (time, depth, longitude) float32 dask.array<chunksize=(292267, 101, 1), meta=np.ndarray>

In [139]:
# del tao_merged.time.encoding["_FillValue"]
# tao_merged.compute().to_zarr("tao_eq_hr_merged_cur.zarr", mode="w", consolidated=True)

tao_gridded.unify_chunks().to_zarr("tao_eq_hr_gridded.zarr", mode="w", consolidated=True)

## TAO in z space: daily avg

In [2]:
tao = pump.obs.read_tao_zarr("ancillary")
tao_clim = tao.mean("time")
tao_clim["eucmax"] = pump.calc.get_euc_max(tao_clim.u)
tao_clim = pump.calc.estimate_euc_depth_terms(tao_clim)
tao_clim.attrs["name"] = "TAO"
tao_clim.load()

NameError: name 'pump' is not defined

## TAO in EUC space

In [9]:
tao = pump.obs.read_tao_zarr("gridded")
tao["Rig"] = pump.calc_tao_ri(tao[["u", "v"]], tao["T"])
tao

,Array,Chunk
Bytes,1.18 GB,8.08 MB
Shape,"(292267, 5, 101)","(10000, 1, 101)"
Count,1616 Tasks,150 Chunks
Type,float64,numpy.ndarray
,Array,Chunk
Bytes,590.38 MB,4.04 MB
Shape,"(292267, 101, 5)","(10000, 101, 1)"
Count,1616 Tasks,150 Chunks
Type,float32,numpy.ndarray
,Array,Chunk


In [243]:
good_data = tao.where((tao.depth < tao.mld) & (tao.deepest < -100) & (shallowest > -15))

interped = dcpy.interpolate.bin_to_new_coord(
    good_data.chunk({"depth": -1}),
    old_coord="depth",
    new_coord="zeuc",
    edges=np.arange(-50, 250, 5),
)

In [248]:
interped.time.encoding.pop("_FillValue")
interped.to_zarr("tao-zeuc.zarr", mode="w", consolidated=True)

## Seasonal mean Ri profile in EUC space

In [ ]:
interped = xr.open_zarr("tao-zeuc.zarr")

In [51]:
grouped = interped.Rig.chunk({"time": -1, "zeuc": 10}).groupby("time.season")

Ri_q = grouped.quantile(dim="time", q=[0.25, 0.5, 0.75]).reindex(
    season=["DJF", "MAM", "JJA", "SON"]
)

Ri_q = Ri_q.assign_coords(num_obs=grouped.count("time"))
Ri_q.zeuc.attrs["long_name"] = "Depth relative to EUC max"
Ri_q.zeuc.attrs["units"] = "m"
Ri_q

/gpfs/u/home/dcherian/python/xarray/xarray/core/common.py:664: FutureWarning: This DataArray contains multi-dimensional coordinates. In the future, the dimension order of these coordinates will be restored as well unless you specify restore_coord_dims=False.
  self, group, squeeze=squeeze, restore_coord_dims=restore_coord_dims


<xarray.DataArray (season: 4, longitude: 5, zeuc: 59, quantile: 3)>
dask.array<getitem, shape=(4, 5, 59, 3), dtype=float64, chunksize=(1, 1, 10, 3), chunktype=numpy.ndarray>
Coordinates:
  * season     (season) object 'DJF' 'MAM' 'JJA' 'SON'
  * quantile   (quantile) float64 0.25 0.5 0.75
  * zeuc       (zeuc) float64 -47.5 -42.5 -37.5 -32.5 ... 232.5 237.5 242.5
  * longitude  (longitude) float32 -204.0 -195.0 -170.0 -140.0 -110.0
    latitude   float32 0.0
    num_obs    (season, longitude, zeuc) int64 dask.array<chunksize=(1, 1, 10), meta=np.ndarray>

In [52]:
Ri_q.load()

<xarray.DataArray (season: 4, longitude: 5, zeuc: 59, quantile: 3)>
array([[[[8.65716985e-01, 1.84699736e+00, 4.07929186e+00],
         [9.68607747e-01, 2.07586669e+00, 4.69831821e+00],
         [1.12714689e+00, 2.45510051e+00, 5.63565819e+00],
         ...,
         [8.25933339e-01, 1.21812980e+00, 2.22266901e+00],
         [3.31773526e-02, 2.15271859e-01, 6.91555336e-01],
         [6.30251313e-01, 9.59637420e-01, 1.28902353e+00]],

        [[6.99998537e-01, 1.29253461e+00, 2.86526357e+00],
         [7.26263211e-01, 1.40190939e+00, 3.03401490e+00],
         [8.14764327e-01, 1.58527315e+00, 3.47211588e+00],
         ...,
         [           nan,            nan,            nan],
         [           nan,            nan,            nan],
         [           nan,            nan,            nan]],

        [[1.04342082e+00, 1.80473934e+00, 3.48219512e+00],
         [1.10594958e+00, 1.94826060e+00, 3.84621979e+00],
         [1.18464712e+00, 2.10952571e+00, 4.29768611e+00],
         ...,
         [           nan,            nan,            nan],
         [           nan,            nan,            nan],
         [           nan,            nan,            nan]],

        [[5.99332633e-01, 1.01832679e+00, 2.02945611e+00],
         [6.36358415e-01, 1.09730253e+00, 2.22929311e+00],
         [6.69077456e-01, 1.17499171e+00, 2.47144998e+00],
         ...,
         [           nan,            nan,            nan],
         [           nan,            nan,            nan],
         [           nan,            nan,            nan]],

        [[3.93460173e-01, 6.78976245e-01, 1.35634816e+00],
         [3.96662435e-01, 6.72044675e-01, 1.30979069e+00],
         [3.89880826e-01, 6.65452753e-01, 1.31105090e+00],
         ...,
         [1.71923451e+00, 2.11843237e+00, 3.14030388e+00],
         [           nan,            nan,            nan],
         [           nan,            nan,            nan]]],


       [[[9.64841693e-01, 2.03641443e+00, 5.24673701e+00],
         [1.00937285e+00, 2.07883897e+00, 5.36984820e+00],
         [1.07954196e+00, 2.17156104e+00, 5.51939592e+00],
         ...,
         [9.45015072e-01, 1.23069288e+00, 3.89244182e+00],
         [3.19059242e-01, 4.46329690e-01, 1.42386922e+00],
         [           nan,            nan,            nan]],

        [[9.77766440e-01, 1.94110422e+00, 4.23775491e+00],
         [1.02809324e+00, 2.09342055e+00, 4.71047287e+00],
         [1.20866875e+00, 2.40437219e+00, 5.50768658e+00],
         ...,
         [           nan,            nan,            nan],
         [           nan,            nan,            nan],
         [           nan,            nan,            nan]],

        [[1.06829041e+00, 1.84754009e+00, 3.53931845e+00],
         [1.11732667e+00, 1.94676703e+00, 3.81204909e+00],
         [1.15610819e+00, 2.03169397e+00, 4.13639554e+00],
         ...,
         [           nan,            nan,            nan],
         [           nan,            nan,            nan],
         [           nan,            nan,            nan]],

        [[6.45954343e-01, 1.04209220e+00, 1.87051596e+00],
         [6.91774334e-01, 1.11851238e+00, 2.00851346e+00],
         [7.23639413e-01, 1.19311723e+00, 2.28014936e+00],
         ...,
         [           nan,            nan,            nan],
         [           nan,            nan,            nan],
         [           nan,            nan,            nan]],

        [[3.90188842e-01, 5.96989693e-01, 1.09097169e+00],
         [3.99915480e-01, 6.17991764e-01, 1.10944998e+00],
         [4.09988742e-01, 6.36251961e-01, 1.14362343e+00],
         ...,
         [           nan,            nan,            nan],
         [           nan,            nan,            nan],
         [           nan,            nan,            nan]]],


       [[[8.05875978e-01, 1.64882518e+00, 3.74725616e+00],
         [8.49498749e-01, 1.81732453e+00, 4.24185284e+00],
         [9.22981317e-01, 1.94493706e+00, 4.58776057e+00],
         ...,
         [3.77534561e-01, 5.81013898e

In [53]:
Ri_q.to_netcdf("tao-hourly-Ri-seasonal-percentiles.nc")

## 20 year Stations to zarr

In [1]:
import ncar_jobqueue

if "client" in locals():
    client.close()
if "cluster" in locals():
    cluster.close()

env = {"OMP_NUM_THREADS": "3", "NUMBA_NUM_THREADS": "3"}

# cluster = distributed.LocalCluster(
#    n_workers=8,
#    threads_per_worker=1,
#    env=env
# )

if "cluster" in locals():
    del cluster

cluster = ncar_jobqueue.NCARCluster(
    project="NCGD0046",
    scheduler_options=dict(dashboard_address=":9898"),
)
# cluster = dask_jobqueue.PBSCluster(
#    cores=9, processes=9, memory="108GB", walltime="02:00:00", project="NCGD0043",
#    env_extra=env,
# )

cluster.scale(18)

In [3]:
import distributed

client = distributed.Client(cluster)
client

Client Scheduler: tcp://10.12.205.44:34054 Dashboard: https://jupyterhub.ucar.edu/dav/user/dcherian/proxy/9898/status,Cluster Workers: 18 Cores: 18 Memory: 450.00 GB


In [2]:
outdir = "/glade/campaign/cgd/oce/people/dcherian/station_zarrs"

In [1]:
import glob
import os
import pandas as pd
import xarray as xr
import zarr

compressor = zarr.Blosc(cname="zstd", clevel=3, shuffle=2)
station_data = (
    "/glade/campaign/cgd/oce/people/bachman/TPOS_1_20_20_year/OUTPUT/STATION_DATA"
)

dirs = glob.glob(f"{station_data}/*")

for stationdirname in dirs:
    fname = stationdirname.split("/")[-1]
    if "zarr" in stationdirname:
        continue
        
    print(f"Processing {stationdirname} ...")
    
    if "_7251" in stationdirname or os.path.exists(f"{outdir}/{fname}.zarr"):
        print("    skipping...")
        continue

    # continue
    
    try:
        station = (
            xr.open_mfdataset(
                f"{stationdirname}/Station_*.nc",
                parallel=True,
                decode_times=False,
                chunks={"longitude": -1, "latitude": 1},
            )
            .sortby("latitude")
            .squeeze()
        )
    except:
        print(f"... FAILED !!!!")

    # TODO: for some reason there are duplicated timestamps near the end
    if ~station.indexes["time"].is_unique:
        dupe = station.indexes["time"].duplicated()
        deduped = station.time.copy(data=~dupe)
        station = station.where(deduped, drop=True)

    # Fix time
    station.time.attrs["long_name"] = ""
    station.time.attrs["units"] = "seconds since 1999-01-01 00:00"
    station = xr.decode_cf(station)
    station["time"] = station.time - pd.Timedelta("7h")

    for var in station.variables:
        station.variables[var].encoding.update({"compressor": compressor})
    
    stop
    station.to_zarr(
        f"{outdir}/{fname}.zarr", mode="w", consolidated=True
    )

Processing /glade/campaign/cgd/oce/people/bachman/TPOS_1_20_20_year/OUTPUT/STATION_DATA/Day_1251-1500 ...


NameError: name 'outdir' is not defined